# Liquid Cell Atlas — Embedding Exploration
**By Nick Semenkovich \<semenko@alum.mit.edu\>**

The goal here is to try storing all CpG sites in a single embedding space (or as a COO sparse tensor) and 
train a model to deconvolute the methylation profiles of the different cell types.

In [2]:
import gc
import os
import random
import sys
import json
import time

In [3]:
import socket
print(f"Running on: {socket.gethostname()}")
print(f"Python version: {sys.version}")

# Require Python 3.11, welcome to the future! :P
if sys.version_info < (3, 11):
    raise RuntimeError("Python 3.11 or greater is required")

# Parallel multithreading limit
NUM_USABLE_CPUS = len(os.sched_getaffinity(0))
print(f"Usable CPUs: {NUM_USABLE_CPUS}")

Running on: acln3
Python version: 3.11.0 | packaged by conda-forge | (main, Jan 14 2023, 12:27:40) [GCC 11.3.0]
Usable CPUs: 40


In [4]:
# External libraries
import matplotlib.pyplot as plt
import numpy as np
# import polars as pl # Pandas but good
import pandas as pd
from scipy.sparse import coo_matrix
# import pybedtools # Blocking Py3.11, using Pandas instead (lol)
# import sklearn.metrics
# import sklearn.model_selection
#import tensorflow as tf
import torch
import torchmetrics
# import tensorflow_decision_forests as tfdf
import zarr

import pysam

import altair as alt
alt.renderers.enable('default')

# We use SeqIO to parse the reference .fa file
from Bio import SeqIO

from tqdm.notebook import tqdm

#import plotly.io as pio
#import plotly.graph_objects as go
#pio.renderers.default = 'colab'

if "google.colab" in sys.modules:
    from google.colab import data_table
    print("Loading pretty datatable formatter.")
    data_table.enable_dataframe_formatter()

In [5]:
# PyTorch version
print(f"PyTorch version: {torch.__version__}")
print(f"PySam version: {pysam.__version__}")

# Tensorflow Decision Forests version
# print(f"Tensorflow Decision Forests version: {tfdf.__version__}")

# Require PyTorch 2.0, welcome to the future! :P
if torch.__version__ < "2.0":
    raise RuntimeError("PyTorch 2.0 or greater is required")

# Set random seeds
RUN_SEED = 42
random.seed(RUN_SEED)
np.random.seed(RUN_SEED)
torch.manual_seed(RUN_SEED)

PyTorch version: 2.0.0
PySam version: 0.20.0


In [6]:
# Run Settings
CHROMOSOMES = ["chr" + str(i) for i in range(1, 23)] + ["chrX", "chrY"]

# For development, forcably re-run parsing even if a cache exists
IGNORE_CACHE = True

In [7]:
# First, we need to see how many CpG sites exist in our genomic annotation.
# We do this flexibly by loading our reference .fa file and counting the number of CpG sites
# We'll later use this to define our embedding size.

# Path to reference .fa file
REFERENCE_FA = "/r5n/liquid-cell-atlas/reference/GRCh38-DAC-U2AF1.fna"


# TODO: Improve caching here, store hash/metadata, read size, etc.

# Maximum read size (our window for padding)
# TODO: Shrink this a little to account for trimming?
READ_SIZE = 150

CACHE_AVAILABLE = os.path.exists("cpg_sites_dict.json") and os.path.exists("chr_lengths.json") and not IGNORE_CACHE

if CACHE_AVAILABLE:
    print("Loading cpg_sites_dict.json from disk")
    with open("cpg_sites_dict.json", "r") as f:
        cpg_sites_dict = json.load(f)
    with open("chr_lengths.json", "r") as f:
        chr_lengths = json.load(f)

    print("Loading cpg_sites_dict.zarr from disk")
    # TODO: Implement -- currentl loads as a zarr Array format, not a dict
    # zarr_cpg_sites_dict = zarr.open("cpg_sites_dict.zarr", mode="r")
    # cpg_sites_dict = {}
    # for k in zarr_cpg_sites_dict:
    #    cpg_sites_dict[k] = zarr_cpg_sites_dict[k]
    #zarr_chr_lengths = zarr.open("chr_lengths.zarr", mode="r")
    #chr_lengths = {}
    #for k in zarr_chr_lengths:
    #    chr_lengths[k] = zarr_chr_lengths[k]
else:
    print("Cache not available, parsing reference .fa file")
    print(f"Loading reference .fa file: {REFERENCE_FA}")

    # We want a list of all CpG sites in the reference genome, including the chromosome and position
    # We'll use this to define our embedding size
    # Each chromosome is identified with a line starting with ">" followed by the chromosome name

    # Store the CpG sites in a dict per chromosome
    cpg_sites_dict = {}
    chr_lengths = {}

    # Iterate over sequences
    for seqrecord in tqdm(SeqIO.parse(REFERENCE_FA, "fasta"), total=len(CHROMOSOMES)):
        if seqrecord.id not in CHROMOSOMES:
            tqdm.write(f"\tSkipping chromosome {seqrecord.id}")
            continue
        # The sequence
        sequence = seqrecord.seq

        # Find all CpG sites
        # The i+1 is because we want to store the 1-based position, because .bed is wild and arguably 1-based maybe:
        # e.g. https://genome-blog.soe.ucsc.edu/blog/2016/12/12/the-ucsc-genome-browser-coordinate-counting-systems/
        # Regardless, $ samtools faidx GRCh38-DAC-U2AF1.fna chr1:0-0 throws an error, while
        # $ samtools faidx GRCh38-DAC-U2AF1.fna chr1:1-1 returns the correct base.
        # cpg_sites_dict[seqrecord.id] = [i+1 for i in range(len(sequence)-1) if sequence[i:i+2]=="CG"]

        cpg_indices = []
        search_str = "CG"
        pos = sequence.find(search_str)

        while pos != -1:
            cpg_indices.append(pos + 1)
            pos = sequence.find(search_str, pos + 1)

        cpg_sites_dict[seqrecord.id] = cpg_indices
        chr_lengths[seqrecord.id] = len(sequence)

    # Save the cpg_sites_dict and chr_lengths to disk as a json
    # TODO: Store this more efficiently as an array using zarr?
    print(f"Saving cpg_sites_dict and chr_lengths to disk as json")
    with open("cpg_sites_dict.json", "w") as f:
        json.dump(cpg_sites_dict, f)
    with open("chr_lengths.json", "w") as f:
        json.dump(chr_lengths, f)

    # TODO: Clean up this trash
    # we need to specify object_codec to store the dict
    zarr_cpg_sites_dict = zarr.open("cpg_sites_dict.zarr", mode="w")
    for k, v in cpg_sites_dict.items():
        zarr_cpg_sites_dict[k] = v
    # Lame. Shove these into one df...
    zarr_chr_lengths = zarr.open("chr_lengths.zarr", mode="w")
    for k, v in chr_lengths.items():
        zarr_chr_lengths[k] = v

Cache not available, parsing reference .fa file
Loading reference .fa file: /r5n/liquid-cell-atlas/reference/GRCh38-DAC-U2AF1.fna


  0%|          | 0/24 [00:00<?, ?it/s]

	Skipping chromosome chrM
	Skipping chromosome chr1_KI270706v1_random
	Skipping chromosome chr1_KI270707v1_random
	Skipping chromosome chr1_KI270708v1_random
	Skipping chromosome chr1_KI270709v1_random
	Skipping chromosome chr1_KI270710v1_random
	Skipping chromosome chr1_KI270711v1_random
	Skipping chromosome chr1_KI270712v1_random
	Skipping chromosome chr1_KI270713v1_random
	Skipping chromosome chr1_KI270714v1_random
	Skipping chromosome chr2_KI270715v1_random
	Skipping chromosome chr2_KI270716v1_random
	Skipping chromosome chr3_GL000221v1_random
	Skipping chromosome chr4_GL000008v2_random
	Skipping chromosome chr5_GL000208v1_random
	Skipping chromosome chr9_KI270717v1_random
	Skipping chromosome chr9_KI270718v1_random
	Skipping chromosome chr9_KI270719v1_random
	Skipping chromosome chr9_KI270720v1_random
	Skipping chromosome chr11_KI270721v1_random
	Skipping chromosome chr14_GL000009v2_random
	Skipping chromosome chr14_GL000225v1_random
	Skipping chromosome chr14_KI270722v1_random
	S

In [8]:
cpg_sites_dict["chr1"][:10]

[10469, 10471, 10484, 10489, 10493, 10497, 10525, 10542, 10563, 10571]

In [9]:
# Write our cpg sites to a .bed
if not os.path.exists("cpg_sites.bed") and not IGNORE_CACHE:
    with open("cpg_sites.bed", "w") as f:
        for chrom in CHROMOSOMES:
            for pos in cpg_sites_dict[chrom]:
                # TODO: Double-check the zero-open half closed bed format
                f.write(f"{chrom}\t{pos}\t{pos+1}\n")

In [10]:
# Now that we parsed our CpG sites, let's print some statistics
# How many CpG sites are there?
total_cpg_sites = sum([len(v) for v in cpg_sites_dict.values()])
print(f"Total CpG sites: {total_cpg_sites:,}")

# Validity check for hg38
assert(total_cpg_sites>28_000_000)

Total CpG sites: 28,254,596


In [11]:
# Count the number of CpGs per chromosome
cpgs_per_chr = {k: len(v) for k, v in cpg_sites_dict.items()}

# Altair chart but don't sort the chromosomes
alt.Chart(pd.DataFrame({"chr": list(cpgs_per_chr.keys()), "cpgs": list(cpgs_per_chr.values())})).mark_bar().encode(
    x=alt.X("chr", sort=None),
    y="cpgs",
)

alt.Chart(...)

In [12]:
# Add up the number of CpGs per chromosome, e.g. chr1, then chr1+chr2, then chr1+chr2+chr3, etc
cpgs_per_chr_cumsum = np.cumsum([cpgs_per_chr[k] for k in CHROMOSOMES])

def embedding_to_genomic_position(embedding_pos):
    """
    Given an embedding position, return the chromosome and position.

    Parameters
    ----------
    embedding_pos : int
        The embedding position.

    Returns
    -------
    chr : str
        The chromosome.
    pos : int
        The position.
    """
    assert embedding_pos >= 0 and embedding_pos < total_cpg_sites

    # Find the index of the first element in cpgs_per_chr_cumsum that is greater than or equal to the embedding position
    chr_index = np.searchsorted(cpgs_per_chr_cumsum, embedding_pos+1, side="left")

    # Now we know the chromosome, but we need to find the position within the chromosome
    # If this is the first chromosome, the position is just the embedding position
    if chr_index == 0:
        return CHROMOSOMES[chr_index], cpg_sites_dict[CHROMOSOMES[chr_index]][embedding_pos]
    # Otherwise, subtract the length of the previous chromosomes from the embedding position
    embedding_pos -= cpgs_per_chr_cumsum[chr_index-1]
    return CHROMOSOMES[chr_index], cpg_sites_dict[CHROMOSOMES[chr_index]][embedding_pos]

# Tests
assert cpgs_per_chr_cumsum[-1] == total_cpg_sites
assert embedding_to_genomic_position(0) == ("chr1", cpg_sites_dict["chr1"][0])
assert embedding_to_genomic_position(1) == ("chr1", cpg_sites_dict["chr1"][1])
# Edges
assert embedding_to_genomic_position(cpgs_per_chr_cumsum[0]) == ("chr2", cpg_sites_dict["chr2"][0])
assert embedding_to_genomic_position(cpgs_per_chr_cumsum[-1]-1) == ("chrY", cpg_sites_dict["chrY"][-1])

In [13]:
def genomic_position_to_embedding(chrom, pos):
    """
    Given a genomic position, return the embedding position.

    Parameters
    ----------
    chrom : str
        The chromosome.
    pos : int
        The position.

    Returns
    -------
    embedding_pos : int
        The embedding position.
    """
    assert chrom in CHROMOSOMES
    
    # Find the index of the chromosome
    chr_index = CHROMOSOMES.index(chrom)
    # Find the index of the CpG site in the chromosome
    cpg_index = cpg_sites_dict[chrom].index(pos)
    # If this is the first chromosome, the embedding position is just the CpG index
    if chr_index == 0:
        return cpg_index
    # Otherwise, add the length of the previous chromosomes to the CpG index
    return cpg_index + cpgs_per_chr_cumsum[chr_index-1]
    
# Tests
assert genomic_position_to_embedding("chr1", cpg_sites_dict["chr1"][0]) == 0
assert genomic_position_to_embedding("chr1", cpg_sites_dict["chr1"][1]) == 1
# Edges
assert genomic_position_to_embedding("chr2", cpg_sites_dict["chr2"][0]) == cpgs_per_chr_cumsum[0]
assert genomic_position_to_embedding("chrY", cpg_sites_dict["chrY"][-1]) == cpgs_per_chr_cumsum[-1]-1

In [14]:
# Let's generate ranges (windows) of all CpGs within READ_SIZE of each other
# This is to subsequently query our .bam/.sam files for reads containing CpGs

# This is a dict of lists, where but each list contains a tuple of CpG ranges witin a window
# The key is the chromosome: e.g. "chr1"
# The value is a list of tuples: e.g. [(0,35), (190,212), (1055,)]
windowed_cpg_sites_dict = {}

# And a reverse dict of dicts where chrom->window_start->[cpgs]
windowed_cpg_sites_dict_reverse = {}

# Loop over all chromosomes
for chrom, cpg_sites in tqdm(cpg_sites_dict.items()):

    windowed_cpg_sites_dict[chrom] = []
    windowed_cpg_sites_dict_reverse[chrom] = {}
    window_start = None
    window_end = None

    # Loop over all CpG sites
    cpg_sites_len = len(cpg_sites)
    temp_per_window_cpg_sites = []
    for i in range(cpg_sites_len):
        temp_per_window_cpg_sites.append(cpg_sites[i])

        if window_start is None:
            window_start = cpg_sites[i]

        # If we're at the end of the chromosome or the next CpG site is too far away
        if i+1 == cpg_sites_len or (cpg_sites[i+1] - cpg_sites[i]) > READ_SIZE:
            # We have a complete window
            window_end = cpg_sites[i]
            windowed_cpg_sites_dict[chrom].append((window_start, window_end))
            windowed_cpg_sites_dict_reverse[chrom][window_start] = temp_per_window_cpg_sites
            temp_per_window_cpg_sites = []
            window_start = None
            window_end = None


  0%|          | 0/24 [00:00<?, ?it/s]

In [15]:
# Write windowed_cpg_sites_dict to a .bed file, which is just a tsv
with open("windowed_cpg_sites_dict.bed", "w") as f:
    for chrom, cpg_sites in windowed_cpg_sites_dict.items():
        for window_start, window_end in cpg_sites:
            f.write(f"{chrom}\t{window_start}\t{window_end}\n")

In [17]:
# Now we'll parse the aligned .bam files and export per-read methylation profiles

# TODO: Break this out into its own script, with arguments:
# - Input bam file
# - Reference genome
# - Window size

# PySam shifts most of the hard work to the C bindings, so it's not too bad.
# I tried a few other options to pre-process data but realistically they're all slow-ish:
#  - Biscuit's epibed/epiread format -- this works OK, but parsing the RLE-encoded string is tricky and not flexible with non-biscuit data (I suspect)
#  - samtools mpileup -- this can't exclude indels, which also makes for annoying parsing

QUALITY_THRESHOLD=20

# One sample bam file for PoC
# HEALTHY_BAM = "/r5n/liquid-cell-atlas/healthy/Healthy_B12_Nu-13_PBMC_emseq/Healthy_B12_Nu-13_PBMC_emseq.bam"
HEALTHY_BAM = "/home/semenko/sorted.tinysam.bam"
print(f"Loading: {os.path.basename(HEALTHY_BAM)}")

# We require an index for performance
input_bam = pysam.AlignmentFile(HEALTHY_BAM, "rb", require_index=True, threads=1)
print(f"Total reads: {input_bam.mapped}")


# Temporary storage for loading into a COO matrix
# For a COO, we want three lists:
# row: the read number (we'll store a read name -> ID dict perhaps?)
# column: cpg #
# data: methylation state (1 [methylated], -1 [unmethylated], and 0 [no data])

next_coo_row_number = 0
read_name_to_row_number = {}

coo_row = []
coo_col = []
coo_data = []


DEBUG = False
PARANOID = False
# DEBUG = False
start_time = time.time()
# This is slow, but we only run it once and store the results for later
for chrom, windowed_cpgs in tqdm(windowed_cpg_sites_dict.items()):
    for start_pos, stop_pos in windowed_cpgs:
        cpgs_within_window = windowed_cpg_sites_dict_reverse[chrom][start_pos]
        if DEBUG: print(f"\tPosition: {start_pos} - {stop_pos}")
        if DEBUG: print(f"\tCovered CpGs: {cpgs_within_window}")
        # cpgs_within_window = [10469]
        # This returns an AlignmentSegment object from PySam
        for aligned_segment in input_bam.fetch(contig=chrom, start=start_pos, end=stop_pos):
            if aligned_segment.mapping_quality < QUALITY_THRESHOLD:
                # if DEBUG: print(f"Skipping read {aligned_segment.query_name} with MAPQ {aligned_segment.mapping_quality} < {QUALITY_THRESHOLD}")
                continue

            if aligned_segment.is_duplicate:
                continue
            if aligned_segment.is_qcfail:
                continue
            if aligned_segment.is_secondary:
                continue

            if DEBUG: print(aligned_segment)

            if PARANOID:
                # Validity tests
                assert aligned_segment.is_mapped
                assert aligned_segment.is_supplementary is False
                assert aligned_segment.reference_start <= stop_pos
                assert aligned_segment.reference_end >= start_pos
                assert aligned_segment.query_alignment_sequence == aligned_segment.get_forward_sequence()

                # Ensure alignment methylation tags exist
                assert aligned_segment.has_tag("XB") # GEM3/Blueprint may add this (also in Biscuit, but different!)
                assert aligned_segment.has_tag("MD") # Location of mismatches (methylation)
                assert aligned_segment.has_tag("YD") # Bisulfite conversion strand label (f: OT/CTOT C->T or r: OB/CTOB G->A)

            # TODO: We ignore paired/unpaired read status for now, and treat them the same
            # Should we treat paired reads / overlapping reads differently?

            bisulfite_parent_strand_is_reverse = None
            if aligned_segment.has_tag("YD"): # Biscuit tag
                yd_tag = aligned_segment.get_tag("YD")
                if yd_tag == "f": # Forward = C→T
                    # This read derives from OT/CTOT strand: C->T substitutions matter (C = methylated, T = unmethylated),
                    bisulfite_parent_strand_is_reverse = False
                elif yd_tag == "r": # Reverse = G→A
                    # This read derives from the OB/CTOB strand: G->A substitutions matter (G = methylated, A = unmethylated)
                    bisulfite_parent_strand_is_reverse = True
            elif aligned_segment.has_tag("XB"): # gem3 / blueprint tag
                raise NotImplementedError("XB tag not validated yet")
                xb_tag = aligned_segment.get_tag("XB") # XB:C = Forward / Reference was CG
                # TODO: Double check one or two of these gem3 tags manually.
                if xb_tag == "C":
                    bisulfite_parent_strand_is_reverse = False
                elif xb_tag == "G": # XB:G = Reverse / Reference was GA
                    bisulfite_parent_strand_is_reverse = True

            # TODO: Think about this old note of mine -- is this correct?
            # We have paired-end reads; one half should (the "parent strand") has the methylation data.
            # The other half (the "daughter strand") was the complement created by PCR, which we don't care about.
            if bisulfite_parent_strand_is_reverse != aligned_segment.is_reverse:
                # Skip if we're not on the bisulfite-converted parent strand.
                if DEBUG: print(f"\t\t Not on methylated strand, skipping: {aligned_segment.query_name}")
                continue

            # get_aligned_pairs returns a list of tuples of (read_pos, ref_pos)
            # We filter this to only include the specific CpG sites from above
            this_segment_cpgs = [e for e in aligned_segment.get_aligned_pairs(matches_only=True) if e[1]+1 in cpgs_within_window]

            # Ok we're on the same strand as the methylation (right?)
            # Let's compare the possible CpGs in this interval to the reference and note status
            #   A methylated C will be *unchanged* and read as C (pair G)
            #   An unmethylated C will be *changed* and read as T (pair A)
            if DEBUG: print(f"bisulfite_parent_strand_is_reverse: {bisulfite_parent_strand_is_reverse}")
            for query_pos, ref_pos in this_segment_cpgs:
                query_base = aligned_segment.query_sequence[query_pos]
                query_base2 = aligned_segment.get_forward_sequence()[query_pos]
                query_base3 = aligned_segment.query_alignment_sequence[query_pos]
                assert query_base == query_base2
                assert query_base == query_base3
                #ref_base = aligned_segment.get_reference_sequence()[ref_pos - aligned_segment.reference_start]
                #assert ref_base.upper() == "C", f"Expected C at {ref_pos} but got {ref_base}"
                # print(f"\t{query_pos} {ref_pos} {query_base} {query_base2} {query_base3} {ref_base}")
                # print(f"\t{query_pos} {ref_pos} C->{query_base}")
                # If query == reference, we're methylated.
                # If query !== reference, we're unmethylated.
                # NOTE: there's an edge where if query != reference AND query == random base, we're an SNV

                # Store in our sparse array
                coo_row.append(next_coo_row_number)
                # instead of ref_pos, we want the index of the CpG in the list of CpGs
                # coo_col.append(ref_pos)
                coo_col.append(genomic_position_to_embedding(chrom, ref_pos+1))
                #coo_data.append(1 if query_base == "C" else 0)
                if query_base == "C":
                    # Methylated
                    coo_data.append(1)
                    if DEBUG: print(f"\t\tMethylated")
                elif query_base == "T":
                    coo_data.append(0)
                    # Unmethylated
                    if DEBUG: print(f"\t\tUnmethylated")
                else:
                    coo_data.append(-1) # or just 0?
                    if DEBUG: print(f"\t\tUnknown!")
                    # raise ValueError(f"Unexpected query base {query_base} at {query_pos} {ref_pos}")

            read_name_to_row_number[aligned_segment.query_name] = next_coo_row_number
            next_coo_row_number += 1

            if DEBUG: print("************************************************\n")

            #query_bp = aligned_segment.query_sequence[pileupread.query_position]
            #reference_bp = aligned_segment.get_reference_sequence()[aligned_segment.reference_start - pileupcolumn.reference_pos].upper()
 
        # break
    # break

print("Elapsed Time: %.2f seconds" % (time.time() - start_time))


Loading: sorted.tinysam.bam
Total reads: 6


  0%|          | 0/24 [00:00<?, ?it/s]

Elapsed Time: 35.33 seconds


In [37]:
my_matrix = coo_matrix((coo_data, (coo_row, coo_col)), shape=(len(read_name_to_row_number), total_cpg_sites)).toarray()
assert(len(my_matrix[0]) == total_cpg_sites)

In [38]:
# Save the matrix, which is an ndarray of shape (n_reads, n_cpgs), to a file
np.save("my_matrix.npy", my_matrix, allow_pickle=True)


# STOP HERE

In [19]:
HEALTHY_BAM = "/r5n/liquid-cell-atlas/healthy/Healthy_B12_Nu-13_PBMC_emseq/Healthy_B12_Nu-13_PBMC_emseq.bam"

QUALITY_THRESHOLD=40

bam_filename_only = os.path.basename(HEALTHY_BAM)
print(f"Loading: {os.path.basename(bam_filename_only)}")


# Temporary storage for loading into a COO matrix
# For a COO, we want three lists:
# row: the read number (we'll store a read name -> ID dict perhaps?)
# column: cpg #
# data: methylation state (1 [methylated], -1 [unmethylated], and 0 [no data])

next_coo_row_number = 0
read_name_to_row_number = {}

coo_row = []
coo_col = []
coo_data = []


# We require an index for performance
input_bam = pysam.AlignmentFile(HEALTHY_BAM, "rb", require_index=True, threads=10)
DEBUG = True
DEBUG = False
# There are two approaches to getting CpGs from PySam -- one is pileup(), and the other is fetch().
# I already had validated pileup() code, so I take that approach here again. It's a little less performant I suspect
# than using fetch(), since we re-query the same reads multiple times, but it's easier to understand and debug.
# I wonder if this is much slower than fetch()?
for cpg_chrom, per_chr_cpg_list in cpg_sites_dict.items():
    print(f"Processing chromosome: {cpg_chrom}")
    for cpg_position in tqdm(per_chr_cpg_list):
        # For each site, grab the pileup (all reads overlapping that exact site)
        # NOTE: A read could be returned by multiple loop iterations (i.e. a ~100bp read could be overlapping multiple CpGs)
        # truncate = return only the *exact* position, not the surrounding overlaps
        # There will only be one turn through this loop; pileup should return only one entry.
        for pileupcolumn in input_bam.pileup(cpg_chrom, start=cpg_position, stop=cpg_position+1,
                                            truncate=True, max_depth=10000, min_mapping_quality=QUALITY_THRESHOLD):
            if DEBUG: print(f"Coverage at CpG: {cpg_position} = {pileupcolumn.nsegments} (unfitered) | {pileupcolumn.get_num_aligned()} (QC filtered)")
            # ?get_query_positions(self)
            # reference_pos = pileupcolumn.reference_pos
            # Loop over all the reads overlapping this exact CpG location
            for pileupread in pileupcolumn.pileups:
                # Skip reads that land on deletions/refskips
                # e.g. This would skip a read that encompasses a CpG, but at the BP of interest, there's a deletion
                if not pileupread.is_del and not pileupread.is_refskip:
                    if DEBUG: print(f"\t Raw read: {pileupread}")
                    
                    assert pileupread.alignment.is_paired is True
                    assert pileupread.alignment.is_proper_pair is True
                    assert pileupread.alignment.is_qcfail is False
                    assert pileupread.alignment.is_unmapped is False
                    assert pileupread.alignment.is_duplicate is False
                    assert pileupread.alignment.has_tag("XB") or pileupread.alignment.has_tag("YD")
                    # assert pileupread.alignment.query_alignment_sequence == pileupread.alignment.query_sequence

                    # We have paired-end reads; one half should (the "parent strand") has the methylation data.
                    # The other half (the "daughter strand") was the complement created by PCR, which we don't care about.
                    # TODO: Let's think about this -- is this correct? 

                    bisulfite_parent_strand_is_reverse = None
                    if pileupread.alignment.has_tag("YD"): # Biscuit tag
                        yd_tag = pileupread.alignment.get_tag("YD")
                        if yd_tag == "f": # Forward = C→T
                            bisulfite_parent_strand_is_reverse = False
                        elif yd_tag == "r": # Reverse = G→A
                            bisulfite_parent_strand_is_reverse = True
                    elif pileupread.alignment.has_tag("XB"): # gem3 / blueprint tag
                        raise NotImplementedError("XB tag not validated yet.")
                        xb_tag = pileupread.alignment.get_tag("XB") # XB:C = Forward / Reference was CG
                        # TODO: Double check one or two of these gem3 tags manually.
                        if xb_tag == "C":
                            bisulfite_parent_strand_is_reverse = False
                        elif xb_tag == "G": # XB:G = Reverse / Reference was GA
                            bisulfite_parent_strand_is_reverse = True

                    # Skip if we're not on the bisulfite-converted parent strand.
                    if bisulfite_parent_strand_is_reverse != pileupread.alignment.is_reverse:
                        if DEBUG: print(f"\t\t Not on methylated strand, skipping.")
                        continue

                    # Ok we're on the same strand as the methylation, let's compare this BP to reference.
                    # Let's refresh how bisulfite actually works:
                    #   A methylated C will be *unchanged* and read as C (pair G)
                    #   An unmethylated C will be *changed* and read as T (pair A)
                    query_bp = pileupread.alignment.query_sequence[pileupread.query_position]
                    # TODO: Shift this up to the level of the pileupcolum?
                    reference_bp = pileupread.alignment.get_reference_sequence()[pileupread.alignment.reference_start - pileupcolumn.reference_pos].upper()
                    
                    # Now we need to populate coo_row, coo_col, and coo_data

                    # coo_row is the read "number" -- since .bam reads aren't assigned a number, we'll assign them one.
                    # We then keep track of the read name to row number mapping in read_name_to_row_number.
                    if pileupread.alignment.query_name in read_name_to_row_number:
                        coo_row.append(read_name_to_row_number[pileupread.alignment.query_name])
                    else:
                        read_name_to_row_number[pileupread.alignment.query_name] = coo_row
                        coo_row.append(next_coo_row_number)
                        next_coo_row_number += 1
                    
                    # coo_col is the CpG site number.
                    coo_col.append(cpg_position)

                    # If query == reference, we're methylated.
                    # If query != reference, we're unmethylated.
                    if query_bp == reference_bp.upper():
                        # If the read is methylated, add the score to the read_score_dict
                        if DEBUG: print(f"\t\t Methylated.")
                        coo_data.append(1)
                    else:
                        # If the read is unmethylated, we don't need to do anything.
                        if DEBUG: print(f"\t\t Unmethylated.")
                        coo_data.append(-1)
                else:
                    if DEBUG: print(f"\t\t Read is a deletion/refskip, skipping.")
            # break
        # break
    break
print("DONE")

Loading: Healthy_B12_Nu-13_PBMC_emseq.bam
Processing chromosome: chr1


  0%|          | 0/2305751 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [111]:
samfile = pysam.AlignmentFile(HEALTHY_BAM, "rb")
# 10468 - 11479
# cpg in this region
cpgs_in_this_region_block = [10468, 10470] # , 10483, 10488, 10492, 10496, 10524, 10541, 10562, 10570]

for pileupcolumn in samfile.pileup("chr1", 10468, 11479, truncate=False, stepper="all", ignore_overlaps=False, max_depth=10000, min_mapping_quality=20, compute_baq=False):
    if pileupcolumn.pos in cpgs_in_this_region_block:
        print("\ncoverage at cpg %s = %s" % (pileupcolumn.pos, pileupcolumn.n))
        for pileupread in pileupcolumn.pileups:
            if not pileupread.is_del and not pileupread.is_refskip:
                print('\tbase in read %s = %s' % (pileupread.alignment.query_name, pileupread.alignment.query_sequence[pileupread.query_position]))

samfile.close()


coverage at cpg 10468 = 67
	base in read A00354:758:HVTT7DSX3:3:1329:15962:20275 = T
	base in read A00354:758:HVTT7DSX3:3:2429:23158:30044 = A
	base in read A00354:758:HVTT7DSX3:3:2322:2202:15107 = A
	base in read A00354:758:HVTT7DSX3:3:1355:4191:3552 = C
	base in read A00354:758:HVTT7DSX3:3:2656:3007:3255 = A
	base in read A00354:758:HVTT7DSX3:3:2408:4824:33708 = G
	base in read A00354:758:HVTT7DSX3:3:2138:14407:2331 = T
	base in read A00354:758:HVTT7DSX3:3:1249:9869:10003 = A
	base in read A00354:758:HVTT7DSX3:3:2361:1940:21637 = T
	base in read A00354:758:HVTT7DSX3:3:2126:16532:20603 = A
	base in read A00354:758:HVTT7DSX3:3:2148:8648:27931 = C
	base in read A00354:758:HVTT7DSX3:3:2355:16975:17300 = T
	base in read A00354:754:HCLY3DMXY:2:2347:25274:22842 = A
	base in read A00354:758:HVTT7DSX3:3:1567:7952:7059 = A
	base in read A00354:758:HVTT7DSX3:3:1448:13792:28635 = T
	base in read A00354:754:HCLY3DMXY:2:2271:18385:1892 = A
	base in read A00354:758:HVTT7DSX3:3:1255:11071:2973 = A


In [147]:
HEALTHY_BAM = "/r5n/liquid-cell-atlas/healthy/Healthy_B12_Nu-13_PBMC_emseq/Healthy_B12_Nu-13_PBMC_emseq.bam"

QUALITY_THRESHOLD=20

bam_filename_only = os.path.basename(HEALTHY_BAM)
print(f"Loading: {os.path.basename(bam_filename_only)}")

# Temporary storage for loading into a COO matrix
# For a COO, we want three lists:
# row: the read number (we'll store a read name -> ID dict perhaps?)
# column: cpg #
# data: methylation state (1 [methylated], -1 [unmethylated], and 0 [no data])

next_coo_row_number = 0
read_name_to_row_number = {}

coo_row = []
coo_col = []
coo_data = []


# We require an index for performance
input_bam = pysam.AlignmentFile(HEALTHY_BAM, "rb", require_index=True, threads=1)

DEBUG = True
DEBUG = False

for cpg_chrom, per_chr_windowed_cpg_list in windowed_cpg_sites_dict.items():
    print(f"Processing windows in chromosome: {cpg_chrom}")
    for start_pos, stop_pos in tqdm(per_chr_windowed_cpg_list):
        for pileupcolumn in input_bam.pileup(cpg_chrom, start=start_pos, stop=stop_pos,
                                            truncate=False, max_depth=10000, min_mapping_quality=QUALITY_THRESHOLD,
                                            stepper="all", ignore_overlaps=False, compute_baq=False):
            # Ignore positions that aren't CpGs
            if pileupcolumn.pos in windowed_cpg_sites_dict_reverse[cpg_chrom][start_pos]:
                if DEBUG: print(f"Coverage at CpG: {pileupcolumn.pos} = {pileupcolumn.nsegments} (unfitered) | {pileupcolumn.get_num_aligned()} (QC filtered)")
                # Loop over all the reads overlapping this single CpG
                for pileupread in pileupcolumn.pileups:
                    continue
                    # Skip reads that land on deletions/refskips
                    # e.g. This would skip a read that encompasses a CpG, but at the BP of interest, there's a deletion
                    if not pileupread.is_del and not pileupread.is_refskip:
                        # print('\tbase in read %s = %s' % (pileupread.alignment.query_name, pileupread.alignment.query_sequence[pileupread.query_position]))
                        # if DEBUG: print(f"\t Raw read: {pileupread}")
                        assert pileupread.alignment.is_paired is True

                        # We can match unpaired reads, it's cool bro
                        # assert pileupread.alignment.is_proper_pair is True

                        assert pileupread.alignment.is_qcfail is False
                        assert pileupread.alignment.is_unmapped is False
                        assert pileupread.alignment.is_duplicate is False
                        assert pileupread.alignment.has_tag("XB") or pileupread.alignment.has_tag("YD")
                        # assert pileupread.alignment.query_alignment_sequence == pileupread.alignment.query_sequence

                        # We have paired-end reads; one half should (the "parent strand") has the methylation data.
                        # The other half (the "daughter strand") was the complement created by PCR, which we don't care about.
                        # TODO: Let's think about this -- is this correct? 

                        bisulfite_parent_strand_is_reverse = None
                        if pileupread.alignment.has_tag("YD"): # Biscuit tag
                            yd_tag = pileupread.alignment.get_tag("YD")
                            if yd_tag == "f": # Forward = C→T
                                bisulfite_parent_strand_is_reverse = False
                            elif yd_tag == "r": # Reverse = G→A
                                bisulfite_parent_strand_is_reverse = True
                        elif pileupread.alignment.has_tag("XB"): # gem3 / blueprint tag
                            raise NotImplementedError("XB tag not validated yet.")
                            xb_tag = pileupread.alignment.get_tag("XB") # XB:C = Forward / Reference was CG
                            # TODO: Double check one or two of these gem3 tags manually.
                            if xb_tag == "C":
                                bisulfite_parent_strand_is_reverse = False
                            elif xb_tag == "G": # XB:G = Reverse / Reference was GA
                                bisulfite_parent_strand_is_reverse = True

                        # Skip if we're not on the bisulfite-converted parent strand.
                        if bisulfite_parent_strand_is_reverse != pileupread.alignment.is_reverse:
                            if DEBUG: print(f"\t\t Not on methylated strand, skipping.")
                            continue

                        # Ok we're on the same strand as the methylation, let's compare this BP to reference.
                        # Let's refresh how bisulfite actually works:
                        #   A methylated C will be *unchanged* and read as C (pair G)
                        #   An unmethylated C will be *changed* and read as T (pair A)
                        query_bp = pileupread.alignment.query_sequence[pileupread.query_position]
                        # TODO: Shift this up to the level of the pileupcolum?
                        reference_bp = pileupread.alignment.get_reference_sequence()[pileupread.alignment.reference_start - pileupcolumn.reference_pos].upper()
                        
                        # Now we need to populate coo_row, coo_col, and coo_data

                        # coo_row is the read "number" -- since .bam reads aren't assigned a number, we'll assign them one.
                        # We then keep track of the read name to row number mapping in read_name_to_row_number.
                        if pileupread.alignment.query_name in read_name_to_row_number:
                            coo_row.append(read_name_to_row_number[pileupread.alignment.query_name])
                        else:
                            read_name_to_row_number[pileupread.alignment.query_name] = coo_row
                            coo_row.append(next_coo_row_number)
                            next_coo_row_number += 1
                        
                        # coo_col is the CpG site number.
                        coo_col.append(cpg_position)

                        # If query == reference, we're methylated.
                        # If query != reference, we're unmethylated.
                        if query_bp == reference_bp.upper():
                            # If the read is methylated, add the score to the read_score_dict
                            if DEBUG: print(f"\t\t Methylated.")
                            coo_data.append(1)
                        else:
                            # If the read is unmethylated, we don't need to do anything.
                            if DEBUG: print(f"\t\t Unmethylated.")
                            coo_data.append(-1)
                    else:
                        if DEBUG: print(f"\t\t Read is a deletion/refskip, skipping.")
        # break
    break
print("DONE")

Loading: Healthy_B12_Nu-13_PBMC_emseq.bam
Processing windows in chromosome: chr1


  0%|          | 0/456706 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [36]:
print(f"{coo_row}")
print(f"{coo_col}")
print(f"{coo_data}")

[0, 1, 2]
[10468, 10468, 10468]
[-1, -1, 1]


In [33]:
pileupread.alignment.query_name

'A00354:754:HCLY3DMXY:2:1482:14235:16783'